In [ ]:
keyword_list = ['이낙연', '이재명', '정세균']

In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )


a = datetime.now()-timedelta(days=7)
a = a.strftime('%Y.%m.%d')
b = datetime.now()-timedelta(days=1)
b = b.strftime('%Y.%m.%d')
print(a, b)

2021.05.14 2021.05.20


In [3]:
print("=" *80)
print("         DAUM NEWS 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\crawler-bot\\{a}-{b}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         DAUM NEWS 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'daum.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
keyword2 = []   # 크롤링 키워드
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
date2 = []      # 게시물 작성 시간

In [6]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
def date_range(start, end):
    start = datetime.strptime(start, "%Y.%m.%d")
    end = datetime.strptime(end, "%Y.%m.%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates
    
dates = date_range(a, b)
print(dates)

['2021.05.14', '2021.05.15', '2021.05.16', '2021.05.17', '2021.05.18', '2021.05.19', '2021.05.20']


In [8]:
# 1. 다음 뉴스 이동


for keyword in keyword_list :
    print("keyword:", keyword)

    for date in dates :
        
        page_num = 1

        while True :

            query_url= (f'https://search.daum.net/search?&at=more&cluster=n&dc=STC&ed={str(date.replace(".",""))}235959&https_on=on&p={page_num}&period=u&pg=1&q={keyword}&r=1&rc=1&sd={str(date.replace(".",""))}000000&sort=recency&w=news')
            driver.get(query_url)
            driver.implicitly_wait(5)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            try :
                item_box = soup.find(name='ul', attrs={'id':'newsResultUL'}).find_all('li')  

                if len(title2) > 0 :            
                    if title2[-len(item_box)] == item_box[0].find(name='div', attrs={'class':'wrap_cont'}).find(name='div',attrs={'class':'wrap_tit mg_tit'}).text :
                        print(f"현재 페이지 {page_num+1}, 현재 키워드 : {keyword} 크롤링을 종료합니다.")
                        break

                for item in item_box :

                    if item.find(name='div', attrs={'class':'wrap_cont'}) :                

                        domain1 = "DaumNews"
                        keyword1 = keyword

                        news = item.find(name='div', attrs={'class':'wrap_cont'})

        #                 print(news)
        #                 break

                        user1 = news.find(name='span', attrs={'class':'f_nb date'}).text.split(" | ")[1]

                        date1 = news.find(name='span', attrs={'class':'f_nb date'}).text.split(" | ")[0]

                        title1 = item.find(name='div',attrs={'class':'wrap_tit mg_tit'}).text
                        
#                         url1 = 

                        print(f'keyword : {keyword1}')
                        print(f'user : {user1}')
                        print(f'date : {date1}')
                        print(f'title : {title1}')

                        print("\n")

                        domain2.append(domain1)
                        keyword2.append(keyword1)
                        date2.append(date1)
                        title2.append(title1)
                        user2.append(user1)



                page_num += 1
            except :
                break


keyword: 정세균
keyword : 정세균
user : 세계로컬신문 
date :  2021.05.14
title :  [포토] 손 꼭 잡은 정세균·권익현 부안군수 


keyword : 정세균
user : KBS
date :  2021.05.14
title :  민주당, 대권 주자 광주 방문 줄이어..민심잡기 치열 


keyword : 정세균
user : 헤럴드경제
date :  2021.05.14
title :  김부겸, 참모진 구성 착수..비서실장 오영식 거론 


keyword : 정세균
user : 프레시안
date :  2021.05.14
title :  [포토] 정세균의 네박자 행보..①당대표→②국회의장→③국무총리... 


keyword : 정세균
user : 프레시안
date :  2021.05.14
title :  [포토] '대통령 꿈' 왔단다..정세균 정읍서 유진섭 시장과 쨍하고... 


keyword : 정세균
user : 광주문화방송 
date :  2021.05.14
title :  이낙연 전 대표, 광주에서 이틀째 대권 행보 


keyword : 정세균
user : 환경일보 
date :  2021.05.14
title :  완도~고흥 해안관광도로 국도 승격 


keyword : 정세균
user : TBS 
date :  2021.05.14
title :  [명랑시사]박용진 “내 역할은 ‘여의도 손흥민’, 경선 규칙 상... 


keyword : 정세균
user : KBS
date :  2021.05.14
title :  민주당, 대권 주자 광주 방문 줄이어..민심잡기 치열 


keyword : 정세균
user : MBC
date :  2021.05.14
title :  [뉴스하이킥] 배준영 "민주당 폭주입법, '단합' 아닌 '담합' 해" 


keyword : 정세균
user : TJB 
date :  2021.05.14
title :  [리포트] 다시 뜨는 '충청대망론', 이번엔? 


keyword : 정세균
u

keyword : 정세균
user : 시사포커스 
date :  2021.05.14
title :  이재명 42%·윤석열 35.1%…李, 영남·강원 外 전 지역서 우세 


keyword : 정세균
user : CNB뉴스 
date :  2021.05.14
title :  여권 잠룡 '빅3', 5·18 맞아 '텃밭' 호남공략 경쟁 


keyword : 정세균
user : 이투데이 
date :  2021.05.14
title :  문재인 대통령, 김부겸 총리ㆍ임혜숙ㆍ노형욱 장관 임명 재가 


keyword : 정세균
user : 뉴스토마토 
date :  2021.05.14
title :  이재명 42% 윤석열 35.1%…가상 양자대결서 이재명 '첫 역전' 


keyword : 정세균
user : YTN
date :  2021.05.14
title :  문 대통령, 김부겸·임혜숙·노형욱 임명안 재가 


keyword : 정세균
user : 한국경제
date :  2021.05.14
title :  대권 양자대결서 첫 역전..이재명 42% vs 윤석열 35.1%[갤럽] 


keyword : 정세균
user : 서울이코노미뉴스 
date :  2021.05.14
title :  여당 대권주자 이낙연과 정세균의 호남 쟁탈전, 누가 웃을까 


keyword : 정세균
user : 주간동아
date :  2021.05.14
title :  "이번엔 성공한다" 또다시 시작된 민주당發 매표 전략 


keyword : 정세균
user : 문화저널21 
date :  2021.05.14
title :  한국갤럽 양자 가상대결서 이재명, 윤석열에 앞서 


keyword : 정세균
user : Queen 
date :  2021.05.14
title :  문대통령, 임혜숙·노형욱 장관 임명안 재가…오늘부터 임기 시작 


keyword : 정세균
user : 이투데이 
date :  2021.05.14
title :  대선 양자대결서 이재명 42% vs 윤석열 35

keyword : 정세균
user : 뉴스1
date :  2021.05.15
title :  전북 샘고을시장 상인들과 간담회 갖는 정세균 전 총리 


keyword : 정세균
user : 동아일보
date :  2021.05.15
title :  "달러 찍는 미국 수준 돼야 '이재명 공약' 지킬 수 있어" 


keyword : 정세균
user : 뉴시스
date :  2021.05.15
title :  로컬푸드 농가 건의사항 듣는 정세균 전 총리 


keyword : 정세균
user : 뉴시스
date :  2021.05.15
title :  인사말 하는 정세균 전 총리 


keyword : 정세균
user : 뉴시스
date :  2021.05.15
title :  정읍 샘고을시장 간담회, 인사말하는 정세균 전 총리 


keyword : 정세균
user : 세계로컬신문 
date :  2021.05.15
title :  [포토] 정세균, 자세 낮추며··· 김제시의원 배려 


keyword : 정세균
user : 미디어펜 
date :  2021.05.15
title :  이재명·이낙연·정세균 '세대결'…민심 잡을 대표 공약은? 


keyword : 정세균
user : 펜앤드마이크 
date :  2021.05.15
title :  쓰면 뱉고 달면 삼키는 문 대통령, 급해지니 이재용 사면 검토 


keyword : 정세균
user : 프레시안
date :  2021.05.15
title :  정세균 "담대한 회복-더 평등한 대한민국..준비된 대통령 필요" 


keyword : 정세균
user : 매일경제
date :  2021.05.15
title :  '1억에 2천 더' '3천에 주거비'..20대 현금지원 쏟아내는 與 잠룡들 


keyword : 정세균
user : 주간동아
date :  2021.05.15
title :  "달러 찍는 미국 수준 돼야 '이재명 공약' 지킬 수 있어" 


keyword : 정세균
user : 서울경제
da

keyword : 정세균
user : 매일경제
date :  2021.05.16
title :  정세균, 총리시절 자문그룹이 외곽서 정책 조언 


keyword : 정세균
user : 국민일보
date :  2021.05.16
title :  4년 전엔 7000명, 지금은 10만명..급성장한 이재명 지지세력 


keyword : 정세균
user : 뉴스1
date :  2021.05.16
title :  정세균 "MB·朴 '범죄인 대통령의 시대' 돌이키지 않겠다" 


keyword : 정세균
user : 더코리아 
date :  2021.05.16
title :  정세균 전 국무총리, 광주·전남 방문 


keyword : 정세균
user : 한겨레
date :  2021.05.16
title :  "호남부터 잡아라" 여권 대선 주자들, 치열한 호남 경쟁 


keyword : 정세균
user : 서울신문
date :  2021.05.16
title :  "나는 왜 그를 택했나"..이재명·이낙연·정세균 '빅3' 최측근이 ... 


keyword : 정세균
user : SBS
date :  2021.05.16
title :  정세균 "결코 이명박 · 박근혜 '범죄인 대통령 시대'로 돌아갈 ... 


keyword : 정세균
user : 뉴스1
date :  2021.05.16
title :  커지는 '경선 연기론'..與주자들은 송영길號 결단 촉구 


keyword : 정세균
user : 한국경제
date :  2021.05.16
title :  "텃밭부터 다지자"..與 대권주자, 5·18 광주行 


keyword : 정세균
user : 한국경제
date :  2021.05.16
title :  정세균 "기업이 중심 돼 제2 新산업 붐 일으켜야..규제 과감히 없... 


keyword : 정세균
user : 서울경제
date :  2021.05.16
title :  정세균 "이명박·박근혜 '범죄인' 시대로 돌이킬 수 없어" 


keyword : 정

keyword : 정세균
user : 뉴스1
date :  2021.05.16
title :  대선 300일, 점차 빨라지는 이재명의 친문행보 


keyword : 정세균
user : 뉴스1
date :  2021.05.16
title :  3박 4일 전북 일정 마무리하는 정세균 전 총리 


keyword : 정세균
user : MBN
date :  2021.05.16
title :  [시사스페셜] 이광재 더불어민주당 의원 "이재용 회장 사면 긍정... 


keyword : 정세균
user : 스카이데일리 
date :  2021.05.16
title :  ‘너도나도 호남行’…與 잠룡들의 텃밭 경쟁 심화 


keyword : 정세균
user : MBN
date :  2021.05.16
title :  [시사스페셜] 이광재 더불어민주당 의원 "내년 대선, 확실한 세대... 


keyword : 정세균
user : 헤럴드경제
date :  2021.05.16
title :  박용진 "'인지도 경선'은 국민 외면..세게 한번 붙자"..맞짱토론 ... 


keyword : 정세균
user : 전라일보 
date :  2021.05.16
title :  "최대 승부처 전북 잡아라"··· 여권 빅3 '세몰이' 광폭 행보 


keyword : 정세균
user : 국제뉴스 
date :  2021.05.16
title :  원전동맹 "원자력 안전교부세 추가해야"…김기현 원내대표 면담 


keyword : 정세균
user : 시민일보 
date :  2021.05.16
title :  여 대선, 이재명 독주에 이낙연 정세균 추격전 양상 


keyword : 정세균
user : 펜앤드마이크 
date :  2021.05.16
title :  ‘김부선’을 ‘김하선’ 만들려는 이재명의 GTX-D 플랜, 대선표... 


keyword : 정세균
user : 위키트리 
date :  2021.05.16
title :  정세균 전 국무총리, 광주·전남 방문 


key

...

keyword : 정세균
user : 노컷뉴스
date :  2021.05.18
title :  與 대권주자·지도부 광주 총출동..5·18 묘역 참배 


keyword : 정세균
user : MBN
date :  2021.05.18
title :  정세균 "검찰·언론개혁이 오늘의 광주항쟁" 


keyword : 정세균
user : 뉴시스
date :  2021.05.18
title :  정세균, 미얀마인과 5·18묘지 참배.."세계인과 광주정신 실천" 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  미얀마 유학생들과 5·18참배하는 정세균 전 총리 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  5·18구묘역 참배하는 정세균 전 총리 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  이한열 열사 어머니 만난 정세균 전 총리 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  박선영 열사 찾은 정세균 전 총리 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  미얀마 유학생과 5·18구묘역 들어서는 정세균 전 총리 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  전두환 기념비 밟는 정세균 전 총리 


keyword : 정세균
user : 이데일리
date :  2021.05.18
title :  광주 집결한 與 유력 주자들 "5·18 정신 계승" 한목소리 


keyword : 정세균
user : 공감신문 
date :  2021.05.18
title :  [공감10주년 딥터뷰] 강병원 “與대선주자들, 정책으로 경쟁...바... 


keyword : 정세균
user : 뉴시스
date :  2021.05.18
title :  "與 대선경선 연기 반대" 65.

keyword : 정세균
user : 서울경제
date :  2021.05.18
title :  "5·18 정신 기려야"..광주 간 與 지도부·대권 도전자들 


keyword : 정세균
user : 스트레이트뉴스 
date :  2021.05.18
title :  [스트레이트 말말말] 정세균 "성폭행범 도주 막은 사람 범죄자 몰... 


keyword : 정세균
user : PD저널 
date :  2021.05.18
title :  41주년 맞은 5‧18...언론, 정치권 광주행에 주목 


keyword : 정세균
user : 세계일보
date :  2021.05.18
title :  與 대선주자들 일제히 광주행.. 호남 민심잡기 나선다 


keyword : 정세균
user : 천지일보 
date :  2021.05.18
title :  與野 대권주자 광주로 총집결… 5.18민주화운동 추모 


keyword : 정세균
user : 뉴스1
date :  2021.05.18
title :  박용진 "민주, 집값 잡으랬더니 종부세 논의..혀 차는 분 많아" 


keyword : 정세균
user : SBS
date :  2021.05.18
title :  민주당 지도부 · 대권주자 광주행..5·18 묘역 참배 


keyword : 정세균
user : 머니S
date :  2021.05.18
title :  문 대통령, 바이든과 첫 정상회담 준비로 5·18 민주화운동 기념... 


keyword : 정세균
user : 문화일보
date :  2021.05.18
title :  민주당 지도부와 대선 주자 5·18 맞아 광주 집결 


keyword : 정세균
user : 신아일보 
date :  2021.05.18
title :  5·18 민주항쟁 41주년… 여야 지도부, 일제히 광주행 


keyword : 정세균
user : 노컷뉴스
date :  2021.05.18
title :  [뉴스쏙:속]최저임금 만원 이뤄질까..文정부 마지막 회의 


key

keyword : 정세균
user : 서울신문
date :  2021.05.19
title :  유시민 "노무현 돌아가시고 처음 꿈꿔..'사랑한다'며 안아드렸다" 


keyword : 정세균
user : 국민일보
date :  2021.05.19
title :  유시민 "꿈에서 본 노무현 대통령, 안아드렸다" 


keyword : 정세균
user : 머니투데이
date :  2021.05.19
title :  盧 12주기 추모전서 정세균 "그 아침 항상 생생" 


keyword : 정세균
user : 천지일보 
date :  2021.05.19
title :  [천지포토] 시민들과 인사 나누는 정세균 전 총리 


keyword : 정세균
user : 세계일보
date :  2021.05.19
title :  與, 대선국면에 쇄신은 '어물쩍' 포퓰리즘은 '적극적' 


keyword : 정세균
user : 뉴스1
date :  2021.05.19
title :  이재명 "앞으로의 길, 盧와 가까워"..정세균 "못이룬 꿈 완수"(종합) 


keyword : 정세균
user : 전남매일 
date :  2021.05.19
title :  광주 찾은 정세균 전 총리 "5월 말 대선 출마 선언" 


keyword : 정세균
user : 광남일보 
date :  2021.05.19
title :  대권주자들 '5월 구상' 호남 민심에 변화올까 


keyword : 정세균
user : 뉴스핌 
date :  2021.05.19
title :  인사 나누는 정세균-이재명 


keyword : 정세균
user : 뉴스핌 
date :  2021.05.19
title :  축사하는 정세균 전 국무총리 


keyword : 정세균
user : 뉴스핌 
date :  2021.05.19
title :  '사람사는 세상전' 참석한 정세균-이재명 


keyword : 정세균
user : 뉴스핌 
date :  2021.05.19
title :  정세균-이재명, '화기

keyword : 정세균
user : 중앙일보
date :  19시간전
title :  식사정치로 세 불린 이재명 "尹 잘 모르겠다, 포장지만 봐서" 


keyword : 정세균
user : 연합뉴스
date :  19시간전
title :  정세균 "검찰, 자정작용 잃은 공룡..더는 가만 있을수 없어" 


keyword : 정세균
user : 제주의소리 
date :  19시간전
title :  제주출신 송재호-오영훈-위성곤, 대선후보 지지 '3인3색' 


keyword : 정세균
user : 대구일보 
date :  19시간전
title :  나경원·이준석 국민의힘 당권도전 


keyword : 정세균
user : 한국경제
date :  19시간전
title :  추미애 "보유세 올려야 집값 잡아"..이광재 "역동적 50대 리더십 ... 


keyword : 정세균
user : 경향신문
date :  20시간전
title :  여야 너도나도 '반도체 일병' 구하기 


keyword : 정세균
user : 전국매일신문 
date :  20시간전
title :  '기본정책' 다진 이재명 '성장키워드'로 중도층 공략 나섰다 


keyword : 정세균
user : 브레이크뉴스 
date :  20시간전
title :  허경영 국가혁명당 명예 대표 “잠룡(潛龍)들 포퓰리즘에 경고” 


keyword : 정세균
user : 조선비즈
date :  20시간전
title :  김동연, 돈 퍼주기식 '현금복지' 비판..기회복지로 새판 짜야 


keyword : 정세균
user : 매일일보 
date :  20시간전
title :  전재수 "휴가철 경선된다" 또 연기 주장 


keyword : 정세균
user : 매일경제TV 
date :  20시간전
title :  "차기 대권 적합도…이재명 25%, 윤석열 19%" 


keyword : 정세균
user : BBS 
date :  20시간전
title :  지속되는 이재용 부회장 '사면' 논쟁... ‘경

keyword : 정세균
user : 뉴스1
date :  2021.05.20
title :  한울삶 방문한 정세균 전 총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  실용 이재명 vs 의리 이낙연 vs 강한 정세균 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  정세균 전 국무총리, 유가협 방문 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문 마친 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문해 대화하는 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 건의사항 듣는 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문해 대화하는 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문해 대화하는 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문해 민주열사 영정 바라보는 정세균 전 총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  유가협 방문한 정세균 전 총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  종로 골목길 걷는 정세균 전 국무총리 


keyword : 정세균
user : 연합뉴스
date :  2021.05.20
title :  종로 골목길 걷는 정세균 전 국무총리 


keyword : 정세균
user : 일요신문 
date :  2021.05.20
tit

In [9]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
df['keyword'] = pd.Series(keyword2)
df['user_text'] = pd.Series(title2)
df['channel'] = pd.Series(user2)
df['date'] = pd.Series(date2)

df

,domain,keyword,uset_text,channel,date
0,DaumNews,정세균,[포토] 손 꼭 잡은 정세균·권익현 부안군수,세계로컬신문,2021.05.14
1,DaumNews,정세균,"민주당, 대권 주자 광주 방문 줄이어..민심잡기 치열",KBS,2021.05.14
2,DaumNews,정세균,"김부겸, 참모진 구성 착수..비서실장 오영식 거론",헤럴드경제,2021.05.14
3,DaumNews,정세균,[포토] 정세균의 네박자 행보..①당대표→②국회의장→③국무총리...,프레시안,2021.05.14
4,DaumNews,정세균,[포토] '대통령 꿈' 왔단다..정세균 정읍서 유진섭 시장과 쨍하고...,프레시안,2021.05.14
...,...,...,...,...,...
1342,DaumNews,정세균,"盧 서거 12주기 전시회에 이재명·정세균..""못 이룬 꿈 잇겠다""",국민일보,2021.05.20
1343,DaumNews,정세균,"여권 대선주자들, 호남 끌어안기 이어 친노 구애전",중앙일보,2021.05.20
1344,DaumNews,정세균,"윤희숙, 이번엔 김부겸의 '아파트 불로소득론' 저격",중앙일보,2021.05.20
1345,DaumNews,정세균,"문 대통령, 홍남기에 하반기 정책 주문…임기 같이하나",강원도민일보,2021.05.20


In [10]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 53.2 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\Qscar KIM\Desktop\crawler-bot\2021.05.14-2021.05.20\\daum.csv 


SystemExit: 프로그램을 종료합니다.

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
